<a href="https://colab.research.google.com/github/adrigrillo/dl/blob/master/assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#    Deep Learning Course 2019
######  Lecturer: Assistant prof. Siamak Mehrkanoon
####  =====================================================================================================

## Assignment-1:

### Download the Iris data and explore it, (visualize it, how many classes, instances, features, etc)
### Select feature number 0 and 2 and the first 100 samples and do the following analysis: 

### 1) Implement the logistic regression model that discussed in the class (with and without regularization). Use Gradient descent algorithm for updating the parameters. Plot the results.

### 2) Explore the influence of learning rate on the convergence of the model. Tune the learning rate.

### 3) Repeat steps 1-4 for different randomly selected features (e.g. 1 and 3 or 2 and 3) and compare the results.

### 4) Give the option to the users to select whatever combination of features they want and your code will do the rest.

### 5)  Try your model on Monk2 dataset and report the results, (test accuracy, training accuracy, optimal learning rate, loss value). Use the last 20% of the data as test.



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df.head()

Describing the data set

In [0]:
df.describe()

In [0]:
print(pd.unique(df[4])) #3 different classes

In [0]:
x1=df.loc[:,0]
y1=df.loc[:,2]

x2=df.loc[:,1]
y2=df.loc[:,3]

x3=df.loc[:,2]
y3=df.loc[:,3]

fig, axs = plt.subplots(3, 1, sharex=True)


# Plot each graph, and manually set the y tick values
axs[0].plot(x1, y1, 'o', color='g')
axs[0].set_ylabel('Feature 2')
axs[0].set_xlabel('Feature 0')

axs[1].plot(x2, y2, 'o',color='skyblue')
axs[1].set_ylabel('Feature 3')
axs[1].set_xlabel('Feature 1')

axs[2].plot(x3, y3,'o')
axs[2].set_ylabel('Feature 3')
axs[2].set_xlabel('Feature 2')

plt.show()

1) Implement the logistic regression model that discussed in the class (with and without regularization). Use Gradient descent algorithm for updating the parameters. Plot the results.

In the first place we define variables b0 and b1 to be features 0 and 2. 

In [0]:
b0=0
b1=2
from sklearn import preprocessing
#Define input
x=df[[b0,b1]].loc[:100].T

#Normalize it
x1=x.loc[0]
x2=x.loc[2]
normalized_X1 = pd.DataFrame(preprocessing.normalize([x1]))
normalized_X2 = pd.DataFrame(preprocessing.normalize([x2]))

x = pd.concat([normalized_X1, normalized_X2])

        0         1         2         3         4         5         6    \
0  0.091991  0.088383  0.084776  0.082972  0.090187  0.097402  0.082972   
0  0.042943  0.042943  0.039876  0.046011  0.042943  0.052145  0.042943   

        7         8         9      ...          91        92        93   \
0  0.090187  0.079364  0.088383    ...     0.110028  0.104617  0.090187   
0  0.046011  0.042943  0.046011    ...     0.141099  0.122695  0.101223   

        94        95        96        97        98        99        100  
0  0.101009  0.102813  0.102813  0.111832  0.091991  0.102813  0.113636  
0  0.128830  0.128830  0.128830  0.131897  0.092021  0.125762  0.184042  

[2 rows x 101 columns]


In [0]:
#Initialize the weights
w= np.random.rand(2) #Size (2,1)
print(w.shape)

#Product of input and weights
z= np.dot(w.T, x) #size(101)
print(z)
sigma=1/(1+np.exp(-z)) 
print(sigma.shape)

#Output
y=np.unique(df[4].loc[:100], return_inverse=True)[1]
print(y.shape)

(2,)
[0.05274893 0.05103899 0.04867581 0.04912729 0.05189396 0.05727355
 0.04847406 0.05254719 0.04676411 0.05169222 0.05596709 0.05149047
 0.05018401 0.04394945 0.0574273  0.05853201 0.05466063 0.05274893
 0.05983847 0.05340216 0.05727355 0.05340216 0.04586114 0.05470863
 0.05345016 0.05320042 0.05320042 0.05425714 0.05360391 0.0506355
 0.05149047 0.05596709 0.05425714 0.05616883 0.05169222 0.0505875
 0.0555156  0.05169222 0.04611088 0.05340216 0.05124073 0.04696586
 0.04611088 0.05320042 0.05601509 0.05018401 0.0540554  0.04847406
 0.05511211 0.05189396 0.09055005 0.08411375 0.09100154 0.07315282
 0.08562195 0.07812892 0.08456523 0.06345036 0.08647693 0.06993467
 0.0656118  0.07787918 0.0774277  0.08285528 0.07139487 0.08602544
 0.07727395 0.07637098 0.0824038  0.07335457 0.08179856 0.07828267
 0.08587169 0.08285528 0.08280728 0.08517046 0.08949334 0.08994482
 0.08069385 0.07159662 0.07184636 0.07119313 0.07506452 0.08461323
 0.075564   0.08069385 0.08798513 0.08260554 0.07466103 0.0

In [0]:
#Cost function

J=np.sum(-y*np.log(sigma)-(1-y)*np.log(1-sigma))/len(sigma)
J

0.6858422243149137

In [0]:
# Learning
alpha=0.9

for i in range(0,1000):
    w=w+alpha*np.dot(y-sigma,x.T)/len(sigma)
    
    #Product of input and weights
    z= np.dot(w.T, x)
    sigma=1/(1+np.exp(-z))
    J=np.sum(-y*np.log(sigma)-(1-y)*np.log(1-sigma))/len(sigma)
    print(J)

0.6357130787121555
0.6354871076294939
0.6352618689127723
0.635037356618953
0.6348135648574699
0.6345904877897551
0.6343681196287727
0.6341464546385539
0.6339254871337369
0.6337052114791115
0.6334856220891683
0.6332667134276496
0.6330484800071073
0.6328309163884615
0.632614017180566
0.6323977770397758
0.6321821906695195
0.6319672528198745
0.6317529582871468
0.6315393019134544
0.6313262785863138
0.6311138832382319
0.6309021108462991
0.6306909564317877
0.630480415059754
0.6302704818386438
0.6300611519198994
0.6298524204975736
0.6296442828079452
0.6294367341291377
0.6292297697807422
0.629023385123443
0.6288175755586484
0.6286123365281218
0.6284076635136187
0.6282035520365261
0.627999997657505
0.6277969959761355
0.6275945426305674
0.6273926332971709
0.6271912636901933
0.6269904295614167
0.6267901266998194
0.6265903509312415
0.6263910981180519
0.6261923641588194
0.6259941449879861
0.6257964365755443
0.625599234926716
0.625402536081636
0.6252063361150361
0.6250106311359348
0.6248154172873275


0.539209864040388
0.5390916774600257
0.5389735287719821
0.5388554179399422
0.5387373449277887
0.5386193096996013
0.5385013122196551
0.5383833524524188
0.5382654303625534
0.538147545914911
0.5380296990745325
0.5379118898066473
0.5377941180766703
0.5376763838502026
0.5375586870930277
0.537441027771112
0.5373234058506022
0.5372058212978249
0.5370882740792838
0.53697076416166
0.5368532915118095
0.536735856096762
0.5366184578837201
0.5365010968400574
0.5363837729333175
0.5362664861312126
0.536149236401622
0.5360320237125915
0.5359148480323309
0.5357977093292143
0.5356806075717775
0.5355635427287172
0.5354465147688903
0.5353295236613116
0.5352125693751537
0.535095651879745
0.534978771144569
0.534861927139263
0.5347451198336163
0.5346283491975699
0.5345116152012152
0.5343949178147919
0.5342782570086888
0.5341616327534403
0.534045045019727
0.5339284937783741
0.5338119790003499
0.5336955006567655
0.5335790587188727
0.5334626531580636
0.5333462839458698
0.5332299510539601
0.5331136544541412
0.53

Creo que se necesita un regularization para recudir el coste más rápido D: